# Covariance Matrix(2)
- - -

#### 1. Covariance Matrix

* Calculating the mean of Sample Vectors - HW7 참고할 것

* Outer product of sample vectors

* Covariance Matrix
- - -

#### pyspark import & SparkContext 생성

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType

sc = SparkContext(master="local[*]")
print(sc)

# Just like using Spark requires having a SparkContext, using SQL requires an SQLContext
sqlContext = SQLContext(sc)
print(sqlContext)

<SparkContext master=local[*] appName=pyspark-shell>


In [2]:
#### 예제 파일 다운로드
from os.path import exists
from google_drive_downloader import GoogleDriveDownloader as gdd
import tarfile

if exists("./NY.tgz"):
    !rm -rf ./NY.tgz
if exists("./NY.parquet"):
    !rm -rf ./NY.parquet
    
gdd.download_file_from_google_drive(file_id='1hAHV6vC6FvVgrYnoN-lR-IfH488-H121',
                                   dest_path = './NY.tgz')
!tar -xzvf NY.tgz
df = sqlContext.read.load("NY.parquet")

NY.parquet/
NY.parquet/_SUCCESS
NY.parquet/part-00022-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00000-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00021-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00001-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00023-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00002-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00024-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00003-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00025-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00004-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00027-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00005-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-0000

In [3]:
# printSchema()와 sample를 이용한 데이터 확인
df.printSchema()
# sample 사용법 참조
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sample
df.sample(False, 0.01).show(3)

root
 |-- Station: string (nullable = true)
 |-- Measurement: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Values: binary (nullable = true)
 |-- dist_coast: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)

+-----------+-----------+----+--------------------+-----------------+-----------------+------------------+------------------+-----+-----------+
|    Station|Measurement|Year|              Values|       dist_coast|         latitude|         longitude|         elevation|state|       name|
+-----------+-----------+----+--------------------+-----------------+-----------------+------------------+------------------+-----+-----------+
|USC00306957|   SNWD_s20|1986|[8B 5E 8F 5E 92 5...|303.5669860839844|44.29610061645508|-74.10279846191406|493.79998779296875|   NY|  RAY BROOK|
|USC00308625|   TOBS_s20|19

### 1. Spark와 Numpy를 사용하여 Covariance Matrix 구해보자!

#### (1) numpy 기초 
아래 참고자료를 활용하여 numpy 기초 학습

* 참고자료 
  * [참고자료 1](http://taewan.kim/post/numpy_cheat_sheet/)
  * [참고자료 2](https://docs.scipy.org/doc/numpy/user/quickstart.html)
  * [참고자료 3](https://scipy-lectures.org/intro/numpy/array_object.html#what-are-numpy-and-numpy-arrays)
  * [참고자료 4](https://doorbw.tistory.com/171)
  * [참고자료 5](https://datascienceschool.net/view-notebook/17608f897087478bbeac096438c716f6/)

* 위 자료에서중에서도 
  * ndarray 생성법
  * vector, matrix 연산
  * 인덱싱 (slicing)
  * 행렬 합치기 (vstack, dstack, hstack)
  * sum, mean
  * np.nan 자료형
  * reshape
  * matmul


* ```numpy``` 명령어 검색
* 예를 들어서 numpy function 중에서 conjugate을 찾는데 정확히 모르겠는 경우 
* ```np.con*?``` 

In [9]:
# np.con* 과 같은 형태의 function 종류를 표시
import numpy as np
np.con*?

In [8]:
# np.* 과 같은 형태의 function 종류를 표시
np.*?

In [8]:
# function 사용하는 방법
help(np.sum)

NameError: name 'np' is not defined

##### A. Numpy를 사용한 배열 생성

    파라미터로 list 객체와 데이터 타입(dtype)을 입력하여 NumPy 배열을 생성합니다. dtype을 생략할 경우, 입력된 list 객체의 요소 타입이 설정됩니다.


In [6]:
import numpy as np
a = np.array([1,2,3], dtype = 'float64')
print(type(a)) # type 출력
print("1차원 입니다 v \n", a) 

b = np.array([[1,2,3], [4,5,6]], dtype = 'int16')
print("\n2차원 입니다 v \n", b) 

c = np.array([[[1,2,3], [4,5,6]], [[3,2,1], [6,5,4]]], dtype = 'float')
print("\n3차원 입니다 v \n", c) 

<class 'numpy.ndarray'>
1차원 입니다 v 
 [1. 2. 3.]

2차원 입니다 v 
 [[1 2 3]
 [4 5 6]]

3차원 입니다 v 
 [[[1. 2. 3.]
  [4. 5. 6.]]

 [[3. 2. 1.]
  [6. 5. 4.]]]


```import matplotlib.pyplot as plt``` 라는 명령어로 그래프 그리는 package를 불러옵니다

In [5]:
import matplotlib.pyplot as plt  # plt 이름은 많이 사용하는 관습임

x = np.linspace(0, 3, 20)
y = np.linspace(0, 9, 20)
#plt.plot(x, y)       # line plot
plt.plot(x, y, 'o') #위 명령어 대신 실행해보세요

In [6]:
# numpy를 python에 적용하기!
np.random.seed(777) # 고정된 random 값을 위한 seed 설정

# np.random.rand : 3개의 피쳐를 가지고 있는 1D 생성, 10만 개의 ndarray 생성
vector_1d = sc.parallelize([np.random.rand(3) for x in range(100000)])
print("★ element의 type 확인 : ", type(vector_1d.takeSample(False, 1)[0]))
print("\n★ sample의 수 : ", vector_1d.count())
print("\n★10만 개의 ndarray 중 무작위 5개 추출 v \n", vector_1d.takeSample(False, 5))


★ element의 type 확인 :  <class 'numpy.ndarray'>

★ sample의 수 :  100000

★10만 개의 ndarray 중 무작위 5개 추출 v 
 [array([0.70252496, 0.4022068 , 0.35872456]), array([0.31798082, 0.85304622, 0.26705958]), array([0.18405346, 0.83202548, 0.68154442]), array([0.11114904, 0.14656309, 0.15327474]), array([0.56958918, 0.71664876, 0.73134317])]


* ```arange()```를 통한 ndarray 생성

In [19]:
C = np.arange(1,5)
print("C:", C)

C: [1 2 3 4]


In [20]:
# 유용한 생성방법
c = np.linspace(0, 1, 6)   # 시작, 끝, 총 길이 즉 0과 1사이에 6개 균일한 포인트 만들기
c

array([0. , 0.2, 0.4, 0.6, 0.8, 1. ])

In [23]:
a = np.ones((3, 3))  # reminder: (3, 3) is a tuple
a

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [24]:
b = np.zeros((2, 2))
b

array([[0., 0.],
       [0., 0.]])

In [30]:
# 단위행렬 만들기
c = np.eye(3)
c

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [32]:
# 입력 array를 대각 element로 하는 행렬 만들기 (대각 이외 element는 모두 '0')
d = np.diag(np.array([1, 2, 3,4]))
d

array([[1, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 4]])

##### B. 전치

* 벡터의 transpose [(전치행렬)](https://ko.wikipedia.org/wiki/%EC%A0%84%EC%B9%98%ED%96%89%EB%A0%AC) : 
본 강의에서 전치행렬 연산은 $\mathbf{x}^T$ 로 표기합니다. 

In [7]:
# in python
import numpy as np

sample1 = np.array([1,2,3]) # 1차원
sample2 = np.array([[1,2,3],[4,5,6]]) # 2차원
print("sample1의 shape : ", sample1.shape)
print("sample2의 shape : ", sample2.shape)

sample1의 shape :  (3,)
sample2의 shape :  (2, 3)


In [8]:
# T와 resahpe를 이용한 전치(행렬에서는 전치를 위해서 reshape 사용하지 말것)
sample1_T = sample1.T
sample1_reshape = sample1.reshape((3,1))

sample2_T = sample2.T
sample2_reshape = sample2.reshape((3,2))

print("sample1의 reshape 전치 : \n", sample1_reshape)
print("sample1의 T 전치(★1차원 array일 경우 T가 적용되지 않는다★) : \n", sample1_T)

print("\nsample2의 reshape 전치 : \n", sample2_reshape)
print("sample2의 T 전치 : \n", sample2_T)


# np.transpose(sample1) 
x = np.arange(4).reshape((2,2))
print('x: \n', x)
print('np.transpose(x): \n', np.transpose(x))

sample1의 reshape 전치 : 
 [[1]
 [2]
 [3]]
sample1의 T 전치(★1차원 array일 경우 T가 적용되지 않는다★) : 
 [1 2 3]

sample2의 reshape 전치 : 
 [[1 2]
 [3 4]
 [5 6]]
sample2의 T 전치 : 
 [[1 4]
 [2 5]
 [3 6]]
x: 
 [[0 1]
 [2 3]]
np.transpose(x): 
 [[0 2]
 [1 3]]


In [9]:
# reshape trick!!!!!
# 피쳐의 개수를 가늠하기 어려울 때, -1를 넣어주면 자동으로 피쳐의 수에 맞춰 전치가 실행된다.
sample1_reshape_trick = sample1.reshape((-1, 1))
sample2_reshape_trick = sample2.reshape((-1, 2))

print("sample1의 reshape(trick) 전치 : \n", sample1_reshape_trick)
print("\nsample2의 reshape(trick) 전치 : \n", sample2_reshape_trick)

sample1의 reshape(trick) 전치 : 
 [[1]
 [2]
 [3]]

sample2의 reshape(trick) 전치 : 
 [[1 2]
 [3 4]
 [5 6]]


* ndarray 차원 추가하기

In [10]:
z = np.array([1, 2, 3])
print("z:", z)

print("z[:, np.newaxis]: \n" , z[:, np.newaxis])
print("z[np.newaxis, :]: \n" , z[np.newaxis, :])

z: [1 2 3]
z[:, np.newaxis]: 
 [[1]
 [2]
 [3]]
z[np.newaxis, :]: 
 [[1 2 3]]


##### C. 행렬의 합 구하기 

1. $(n\times d)$ 길이를 갖는 행렬 $A$ 와 행렬 $B$의 합을 구한다

  * ```>> A+B```
2. ```dstack```을 활용하여 더하는 방법
  * ``` 
    >>> C = np.dstack([A, B]) 
    >>> print(C)
    ```  
  * ```np.sum(C, axis=2)```
  
3. **1, 2의 방법의 return 값을 확인할 것!!!**


* `+` 를 이용한 행렬의 합 구하기

In [11]:
A = np.array([1,2,3,4,5])
B = np.array([10,20,30,40,50])

print("A : ", A)
print("B : ", B)
print("A + B : ", A + B)

A :  [1 2 3 4 5]
B :  [10 20 30 40 50]
A + B :  [11 22 33 44 55]


* `sum` 를 이용한 행렬의 합 구하기
* ```np.sum(array, axis= sum하고자하는 차원입력)```

In [12]:
D=C.reshape(2,2)
print("D:", D)
E=np.sum(D,axis=0)
F=np.sum(D,axis=1)
print("axis=0 sum:", E)

print("axis=1 sum:", F)
F

NameError: name 'C' is not defined

* ``np.dstack``과 ``np.sum``을 이용한 행렬의 합 구하기

In [13]:
C = np.dstack([A, B])

print("np.dstack([A,B]): ")
print(C)
print("\nnp.dstack, np.sum : ", np.sum(C, axis=2))

np.dstack([A,B]): 
[[[ 1 10]
  [ 2 20]
  [ 3 30]
  [ 4 40]
  [ 5 50]]]

np.dstack, np.sum :  [[11 22 33 44 55]]


#### (2)  Outer product of sample vectors 
우선 행렬의 기본적 [설명](https://ko.wikipedia.org/wiki/%ED%96%89%EB%A0%AC)을 참고바랍니다. 

다음은 벡터의 [외적](https://ko.wikipedia.org/wiki/%EC%99%B8%EC%A0%81)을 정의합니다. 벡터 $\mathbf{v}$의 외적은
$$
\text{Outer product} = \mathbf{v}^T \mathbf{v}
$$
* 본 강의에서는 기본 벡터 표기를 (특별히 언급하지 않는한) row vector로 사용합니다.
  * 즉 $\mathbf{v}$는 row vector (행벡터), $\mathbf{v}^T$는 column vector (열벡터)
* 위에서 외적을 설명하는 위키 링크 문서의 경우 $\mathbf{v}$의 기본 표기는 column vector로 가정합니다. 이처럼, 자료를 참고할때, 기본적으로 벡터의 모양이 어떤 형태로 되어 있는지 확인하고 공부를 하셔야 합니다.  

<br>

* **아래의 numpy.outer의 Parameter와 return을 확인해볼 것**

<p><img alt="" src="https://drive.google.com/uc?id=1dzT9jia2x3pnsXoi4Ig-q3RAeoxptvnv" style="height:324px; width:400px" /></p>

In [7]:
print("numpy의 np.outer을 사용한 sample1 outer product : \n", 
      np.outer(sample1, sample1))
print("numpy의 np.outer을 사용한 sample1 outer product : \n", 
      np.outer(sample1_reshape_trick, sample1))


print("\nnumpy의 np.outer을 사용한 sample2 outer product : \n", 
      np.outer(sample2, sample2))
print("\nnumpy의 np.outer을 사용한 sample2 outer product : \n", 
      np.outer(sample2_reshape_trick, sample2))

NameError: name 'sample1' is not defined

### Excercise 1 - Outer product of sample vectors (20 point)

- - -

다음 데이터에 대하여 다음 과제를 수행하세요.

- regular.csv : KBO에서 활약한 타자들의 역대 정규시즌 성적을 포함하여 몸무게, 키 ,생년월일 등의 기본정보

**위의 두 데이터는 모두 `,`로 구분되어 있습니다.**

 - **데이터의 자세한 설명은 다음의 링크를 참조해주세요.([여기를 눌러서 데이터 설명 참고](https://dacon.io/cpt6/62885))**
 - 또한 regular.csv를 직접 열어서 데이터가 어떻게 저장되어 있는지 확인해주세요.

**task**

* 1 : 주어지는 code는 ``regular.csv``에 특정 기준을 적용하여 생성되는 RDD입니다. ``생성되는 RDD가 어떤 RDD인지 자세하게 2 ~ 3줄 이내로 설명``하세요. (5 point)

<br>

* 2 : 

    ``RDD_to_outer`` 함수는 ``ndarray``를 ``parameter``를 받아 ``outer product를 return`` 합니다.(10 point)

<br>

* 3 : ``task1에서 주어진 RDD``에 ``map``을 이용하여 ``RDD_to_outer``를 적용합니다. (10 point)

**task**

* 1 : 주어지는 code는 ``regular.csv``에 특정 기준을 적용하여 생성되는 RDD입니다. ``생성되는 RDD가 어떤 RDD인지 자세하게 2 ~ 3줄 이내로 설명``하세요. (5 point)

In [10]:
import urllib.request
import re

f = urllib.request.urlretrieve ("https://docs.google.com/uc?export=download&id=1b_L-rJYJC9Oqga0fQ2zh2M763CTM8jzR", "regular.csv")
regular = sc.textFile("./regular.csv").map(lambda x : x.split(","))

def stringToFloat(element): # G R H RBI BB
    float_list = []
    for index in [5,7,8,13,16]:
        float_list.append(float(element[index]) if element[index] != '' else float(0))
    return (int(element[0]), np.array(float_list, dtype = 'float64'))

RDD_step1 = regular.filter(lambda x : x[3] == "두산").map(stringToFloat).cache()
RDD_step2 = RDD_step1.reduceByKey(lambda x, y : x + y).sortByKey().map(lambda x : x[1]).cache()

#### task 1 답안 작성:먼저 G,R,H,RBI,BB가 있는 행만 전부 뽑아내고 데이터 타입은 float64로 변경을 해주고 0일경우는 인덱스에 추가하지 않으며 3번째에 있는 값이 두산인 경우만 문자에서 float형태로 바꿔주고 reduceByKey를 통해 같은 key의 value값들을 더해주고 정렬후 모든 값들을 x[1]번째에 넣어줌.
여기에 입력하시오!!!

**task**

* 2 : 

    ``RDD_to_outer`` 함수는 ``ndarray``를 ``parameter``를 받아 ``outer product를 return`` 합니다.(10 point)

<br>

* 3 : ``task1에서 주어진 RDD``에 ``map``을 이용하여 ``RDD_to_outer``를 적용합니다. (10 point)

```
# task 1-3 output
[array([[63504., 18900., 26460., 13608., 13104.],
        [18900.,  5625.,  7875.,  4050.,  3900.],
        [26460.,  7875., 11025.,  5670.,  5460.],
        [13608.,  4050.,  5670.,  2916.,  2808.],
        [13104.,  3900.,  5460.,  2808.,  2704.]]),
 array([[772641., 375333., 510699., 256668., 244362.],
        [375333., 182329., 248087., 124684., 118706.],
        [510699., 248087., 337561., 169652., 161518.],
        [256668., 124684., 169652.,  85264.,  81176.],
        [244362., 118706., 161518.,  81176.,  77284.]])]
```

In [11]:
print(RDD_step2.collect())

[array([252.,  75., 105.,  54.,  52.]), array([879., 427., 581., 292., 278.]), array([143.,  53.,  72.,  47.,  31.]), array([1500.,  782., 1589., 1008.,  785.]), array([36., 12., 10.,  4.,  2.]), array([38., 12.,  6.,  2.,  5.]), array([15.,  1.,  3.,  5.,  0.]), array([91., 21., 36., 17., 14.]), array([1204.,  461.,  818.,  428.,  363.]), array([574., 368., 601., 416., 249.]), array([1131.,  660., 1294.,  771.,  597.]), array([373.,  76., 134.,  61.,  43.]), array([76., 30., 67., 50., 29.]), array([359., 149., 174.,  64.,  60.]), array([1096.,  578.,  933.,  444.,  284.]), array([544., 316., 577., 281., 124.]), array([297., 109., 149.,  74.,  60.]), array([3., 0., 1., 0., 0.]), array([23.,  7.,  6.,  4.,  5.]), array([953., 354., 781., 345., 286.]), array([50., 10., 15.,  9.,  4.]), array([1066.,  474.,  980.,  547.,  348.]), array([58., 11.,  6.,  3.,  3.]), array([1325.,  615., 1070.,  467.,  414.]), array([639., 275., 520., 368., 247.]), array([304.,  50., 112.,  33.,  29.]), array

In [12]:
# 1 - 2 답안 작성
def RDD_to_outer(ndarray):#RDD_to_outer 함수에 ndarray라는 RDD를 직접 받음
    outerproduct=np.outer(ndarray,ndarray) #outerproduct라는 이름에 np.outer함수를 사용하고 받은 함수 인자를 넣어줘서 외적값을 저장함
    return outerproduct#결과값을 반환함
    # 여기에 작성

# 1 - 3 답안 작성
task3_result=RDD_step2.map(lambda x:RDD_to_outer(x))#task3_result는 RDD_step2의 map을 이용하여 RDD_to_outer함수를 사용하고 값을 저장함
# output
task3_result.take(2)#결과값 2개만 출력

[array([[63504., 18900., 26460., 13608., 13104.],
        [18900.,  5625.,  7875.,  4050.,  3900.],
        [26460.,  7875., 11025.,  5670.,  5460.],
        [13608.,  4050.,  5670.,  2916.,  2808.],
        [13104.,  3900.,  5460.,  2808.,  2704.]]),
 array([[772641., 375333., 510699., 256668., 244362.],
        [375333., 182329., 248087., 124684., 118706.],
        [510699., 248087., 337561., 169652., 161518.],
        [256668., 124684., 169652.,  85264.,  81176.],
        [244362., 118706., 161518.,  81176.,  77284.]])]

### 1. 공분산 구하기 첫번째 방법
1. Sample vector $\mathbf{x}_i$, $i=1,\ldots, n$와 평균 벡터 $\bar{\mathbf{x}}$ 에 대해서 다음 연산을 수행합니다. 
2. $\tilde{\mathbf{x}}_i = \mathbf{x}_i-\bar{\mathbf{x}}$
3. 각 벡터 $\tilde{\mathbf{x}}_i$의 outer product (외적)을 구합니다
\begin{align*}
\tilde{\mathbf{O}}_i = \tilde{\mathbf{x}}_i^T \tilde{\mathbf{x}}_i
\end{align*}
4. Sample covariance matrix는 다음과 같이 정의됩니다!
\begin{align*}
\text{Cov}_1(\mathbf{x}) = \frac{1}{n} \sum_{i=1}^n \tilde{\mathbf{O}}_i
\end{align*}

In [11]:
sample1 = np.array([1,2])
sample2 = np.array([3,4])
sample3 =np.array([5,6])
sample_sum=np.dstack([sample1,sample2,sample3])
sample_max=np.sum(np.dstack([sample1,sample2]),axis=1)
print(sample_sum)
print(sample_max)

[[[1 3 5]
  [2 4 6]]]
[[3 7]]


In [258]:
a=[1,2,3]
b=[4,5,6]
sample_easy=np.sum(np.dstack([a,b]),axis=0)
print(sample_easy)

[[1 4]
 [2 5]
 [3 6]]


In [13]:
# 위 절차에 맞추어서 공분산 1 번째 방법으로 구하기 
# in python
import numpy as np

sample1 = np.array([1,2,3])
sample2 = np.array([4,5,6])
sample3 = np.array([7,8,9])

print("각 sample의 값 : ", sample1, sample2, sample3)

# 평균 벡터 구하기
sample_sum = np.dstack([sample1, sample2,sample3])
print(sample_sum)
sample_sum = np.sum(np.dstack([sample_sum, sample3]), axis = 2)
print(sample_sum)
sample_mean = sample_sum / 3

# 평균이 빼진 각 sample의 outer product 구하기
# sample에서 평균 벡터 빼기
sample1_del_mean = sample1 - sample_mean
sample2_del_mean = sample2 - sample_mean
sample3_del_mean = sample3 - sample_mean

# outerproduct 구하기
sample1_outer = np.outer(sample1_del_mean,sample1_del_mean)
sample2_outer = np.outer(sample2_del_mean,sample2_del_mean)
sample3_outer = np.outer(sample3_del_mean,sample3_del_mean)

# Sample covariance matrix 구하기
sample_cov = np.dstack([sample1_outer, sample2_outer])
sample_cov = np.sum(np.dstack([sample_cov, sample3_outer]), axis = 2)/3

print("\nSample covariance matrix : ")
print(sample_cov)

각 sample의 값 :  [1 2 3] [4 5 6] [7 8 9]
[[[1 4 7]
  [2 5 8]
  [3 6 9]]]


NameError: name 'samole_sum' is not defined

In [0]:
# in spark
vector_list = sc.parallelize([np.array([1,2,3]),np.array([4,5,6]),np.array([7,8,9])])
print("각 sample의 값 : ", vector_list.collect())

# 평균 벡터 구하기
sample_sum = vector_list\
    .reduce(lambda ndarr1, ndarr2 : np.sum(np.dstack([ndarr1, ndarr2]), axis=2))
sample_mean = sample_sum / vector_list.count()

# 평균이 빼진 각 sample의 outer  구하기
# sample에서 평균 벡터 빼기 및 outerproproductduct
vector_outer = vector_list.map(lambda x : x - sample_mean)\
                          .map(lambda x : np.outer(x, x)).cache()

# Sample covariance matrix
sample_cov = vector_outer.reduce(lambda x, y : np.sum(np.dstack([x, y]), axis=2))
print("\nSample covariance matrix : ")
print(sample_cov / vector_list.count())


각 sample의 값 :  [array([1, 2, 3]), array([4, 5, 6]), array([7, 8, 9])]

Sample covariance matrix : 
[[6. 6. 6.]
 [6. 6. 6.]
 [6. 6. 6.]]



### Excercise 2 - 
#### 공분산 구하기 첫 번째 방법을 적용해 보자! (25 point)

- - -

``Excercise1 에서 주어진 RDD_step2를 이용하여 진행``합니다.

- - -

**task**
1. $\bar{\mathbf{x}} = \frac{1}{n}\sum_{i=1}^n \mathbf{x}_i$ (평균 벡터 구하기) (5 point)
    - `평균 벡터 구하는 방법은 HW7를 참조합시다.!`
2. $\tilde{\mathbf{x}}_i = \mathbf{x}_i-\bar{\mathbf{x}}$ (sample에서 평균 벡터 빼기) (5 point)
3. 각 벡터 $\tilde{\mathbf{x}}_i$의 outer product (외적)을 구하기 (5 point)
\begin{align*}
\tilde{\mathbf{O}}_i = \tilde{\mathbf{x}}_i^T \tilde{\mathbf{x}}_i
\end{align*}
4. Sample covariance matrix (10 point)
\begin{align*}
\text{Cov}_1(\mathbf{x}) = \frac{1}{n} \sum_{i=1}^n \tilde{\mathbf{O}}_i
\end{align*}

In [25]:
RDD_step2

[array([252.,  75., 105.,  54.,  52.]),
 array([879., 427., 581., 292., 278.]),
 array([143.,  53.,  72.,  47.,  31.]),
 array([1500.,  782., 1589., 1008.,  785.]),
 array([36., 12., 10.,  4.,  2.]),
 array([38., 12.,  6.,  2.,  5.]),
 array([15.,  1.,  3.,  5.,  0.]),
 array([91., 21., 36., 17., 14.]),
 array([1204.,  461.,  818.,  428.,  363.]),
 array([574., 368., 601., 416., 249.]),
 array([1131.,  660., 1294.,  771.,  597.]),
 array([373.,  76., 134.,  61.,  43.]),
 array([76., 30., 67., 50., 29.]),
 array([359., 149., 174.,  64.,  60.]),
 array([1096.,  578.,  933.,  444.,  284.]),
 array([544., 316., 577., 281., 124.]),
 array([297., 109., 149.,  74.,  60.]),
 array([3., 0., 1., 0., 0.]),
 array([23.,  7.,  6.,  4.,  5.]),
 array([953., 354., 781., 345., 286.]),
 array([50., 10., 15.,  9.,  4.]),
 array([1066.,  474.,  980.,  547.,  348.]),
 array([58., 11.,  6.,  3.,  3.]),
 array([1325.,  615., 1070.,  467.,  414.]),
 array([639., 275., 520., 368., 247.]),
 array([304.,  50., 

**task**
1. $\bar{\mathbf{x}} = \frac{1}{n}\sum_{i=1}^n \mathbf{x}_i$ (평균 벡터 구하기) (5 point)

```
Exercise 1에서 생성한 RDD_step2 를 이용합니다.
 PythonRDD[86] at RDD at PythonRDD.scala:53 
 
# task 2-1 output
sample sum :  [[22250.  9689. 17891.  9259.  6733.]]
sample count :  52
sample mean :  [[427.88461538 186.32692308 344.05769231 178.05769231 129.48076923]]
```

In [132]:
print("Exercise 1에서 생성한 RDD_step2 를 이용합니다.\n",RDD_step2,"\n")
sample_sum=RDD_step2.reduce(lambda x,y: np.sum(np.dstack([x+y]),axis=2))#sample_sum에 RDD_step2를 reduce하여 열끼리 더하여 저장
sample_mean=sample_sum/RDD_step2.count()#평균값mean은 다 더한 것을 count갯수로 나눔

# 2 - 1 답안 작성





# output
print("sample sum : ", sample_sum)#sum출력
print("sample count : ", RDD_step2.count())#count출력
print("sample mean : ", sample_mean)#mean출력

Exercise 1에서 생성한 RDD_step2 를 이용합니다.
 PythonRDD[35] at RDD at PythonRDD.scala:53 

sample sum :  [[22250.  9689. 17891.  9259.  6733.]]
sample count :  52
sample mean :  [[427.88461538 186.32692308 344.05769231 178.05769231 129.48076923]]


**task**
2. $\tilde{\mathbf{x}}_i = \mathbf{x}_i-\bar{\mathbf{x}}$ (sample에서 평균 벡터 빼기) (5 point)

<br>

3. 각 벡터 $\tilde{\mathbf{x}}_i$의 outer product (외적)을 구하기 (5 point)
\begin{align*}
\tilde{\mathbf{O}}_i = \tilde{\mathbf{x}}_i^T \tilde{\mathbf{x}}_i
\end{align*}

```
# task 2-3 output 예시(출력 결과는 다를 수 있습니다)
[[30935.39792899 19580.69304734 42046.57026627 21819.83949704
  13627.67529586]
 [19580.69304734 12393.68380178 26613.55732249 13810.96116864
   8625.69563609]
 [42046.57026627 26613.55732249 57148.58025148 29656.94563609
  18522.37389053]
 [21819.83949704 13810.96116864 29656.94563609 15390.31102071
   9612.08542899]
 [13627.67529586  8625.69563609 18522.37389053  9612.08542899
   6003.26960059]]
 ```

In [62]:
# task 2-2와 2-3 답안 작성


# output
print("# task 3 output 예시(출력 결과는 다를 수 있습니다)")
a=RDD_step2.map(lambda x:(x-sample_mean)).map(lambda x:np.outer(x,x)).reduce(lambda x,y:(x+y))#a에 RDD_step2의 map을 하여 x-sample_mean의 값들로 리스트를 만들고 map을 한 번 더하여 나온 값들에 외적을 하고 reduce하여 전부 더해줌
print(a)#a값을 출력


# task 3 output 예시(출력 결과는 다를 수 있습니다)
[[9944047.30769231 4797399.96153846 9322086.34615385 4921052.34615385
  3665524.88461538]
 [4797399.96153846 2462705.44230769 4680484.01923077 2466252.01923077
  1854649.82692308]
 [9322086.34615385 4680484.01923077 9249786.82692308 4968547.82692308
  3653480.55769231]
 [4921052.34615385 2466252.01923077 4968547.82692308 2814554.82692308
  2030642.55769231]
 [3665524.88461538 1854649.82692308 3653480.55769231 2030642.55769231
  1541260.98076923]]


**task**
4. Sample covariance matrix
\begin{align*}
\text{Cov}_1(\mathbf{x}) = \frac{1}{n} \sum_{i=1}^n \tilde{\mathbf{O}}_i
\end{align*}

```
# task 2-4 output
Sample covariance matrix 방법1 결과(동일한 결과가 출력되어야 합니다): 
[[191231.67899408  92257.69156805 179270.89127219  94635.62204142
   70490.86316568]
 [ 92257.69156805  47359.72004438  90009.30806213  47427.92344675
   35666.34282544]
 [179270.89127219  90009.30806213 177880.51590237  95548.9966716
   70259.24149408]
 [ 94635.62204142  47427.92344675  95548.9966716   54126.05436391
   39050.81841716]
 [ 70490.86316568  35666.34282544  70259.24149408  39050.81841716
   29639.63424556]]
   ```

In [65]:
# 2-4 답안 작성
cov1 = a/RDD_step2.count()#나온 값에 count를 나눠주고 cov1에 넣어줌
# output
print("Sample covariance matrix 방법1 결과(동일한 결과가 출력되어야 합니다): ")
print(cov1)#cov1을 출력

Sample covariance matrix 방법1 결과(동일한 결과가 출력되어야 합니다): 
[[191231.67899408  92257.69156805 179270.89127219  94635.62204142
   70490.86316568]
 [ 92257.69156805  47359.72004438  90009.30806213  47427.92344675
   35666.34282544]
 [179270.89127219  90009.30806213 177880.51590237  95548.9966716
   70259.24149408]
 [ 94635.62204142  47427.92344675  95548.9966716   54126.05436391
   39050.81841716]
 [ 70490.86316568  35666.34282544  70259.24149408  39050.81841716
   29639.63424556]]


### Excercise 3 - 
#### 공분산 구하기 두 번째 방법을 적용해 보자! (25 point)

- - -

``Excercise1 에서 주어진 RDD_step2를 이용하여 진행``합니다.

- - -
#### 공분산 구하기 두번째 방법

1. Sample vector $\mathbf{x}_i$, $i=1,\ldots, n$와 평균 벡터 $\bar{\mathbf{x}}$ 에 대해서 다음 연산을 수행합니다. 
2. $\tilde{\mathbf{x}}_i = \mathbf{x}_i-\bar{\mathbf{x}}$
3. 다음과 같이 행렬을 만듭니다 (10 point)
\begin{align*}
\mathbf{X} = 
\left[
\begin{array}{c}
\tilde{\mathbf{x}}_1 \\
\tilde{\mathbf{x}}_2 \\
\vdots \\
\tilde{\mathbf{x}}_{n-1} \\
\tilde{\mathbf{x}}_{n} 
\end{array}
\right] = 
\left[
\begin{array}{cccc}
x_{11} - \bar{x}_{11} & x_{12} - \bar{x}_{12} & \cdots & x_{1d} - \bar{x}_{1d} \\
x_{21} - \bar{x}_{21} & x_{22} - \bar{x}_{22} & \cdots & x_{2d} - \bar{x}_{2d} \\
\vdots & &\ddots & \vdots \\
x_{n1} - \bar{x}_{n1} & x_{n2} - \bar{x}_{n2} & \cdots & x_{nd} - \bar{x}_{nd} \\
\end{array}
\right]
\end{align*}

<br>

4. Sample covariance matrix는 다음과 같이 정의됩니다(15 point)
    - 행렬곱을 진행할 때, ``.T`` 또는 ``reshape``와 ``np.matmul``을 사용할 것!
\begin{align*}
\text{Cov}_2(\mathbf{x}) &= \frac{1}{n}\mathbf{X}^T \mathbf{X} \\
&= \left[
\begin{array}{cccc}
x_{11} - \bar{x}_{11} & x_{21} - \bar{x}_{21} & \cdots & x_{n1} - \bar{x}_{n1} \\
x_{12} - \bar{x}_{12} & x_{22} - \bar{x}_{22} & \cdots & x_{n2} - \bar{x}_{n2} \\
\vdots & &\ddots & \vdots \\
x_{1d} - \bar{x}_{1d} & x_{2d} - \bar{x}_{2d} & \cdots & x_{nd} - \bar{x}_{nd} \\
\end{array}
\right]\left[
\begin{array}{cccc}
x_{11} - \bar{x}_{11} & x_{12} - \bar{x}_{12} & \cdots & x_{1d} - \bar{x}_{1d} \\
x_{21} - \bar{x}_{21} & x_{22} - \bar{x}_{22} & \cdots & x_{2d} - \bar{x}_{2d} \\
\vdots & &\ddots & \vdots \\
x_{n1} - \bar{x}_{n1} & x_{n2} - \bar{x}_{n2} & \cdots & x_{nd} - \bar{x}_{nd} \\
\end{array}
\right]
\end{align*}

**task**
1. Sample vector $\mathbf{x}_i$, $i=1,\ldots, n$와 평균 벡터 $\bar{\mathbf{x}}$ 에 대해서 다음 연산을 수행합니다. 
2. $\tilde{\mathbf{x}}_i = \mathbf{x}_i-\bar{\mathbf{x}}$
3. 다음과 같이 행렬을 만듭니다( **np.vstack 이용할 것** ) (10 point)
\begin{align*}
\mathbf{X} = 
\left[
\begin{array}{c}
\tilde{\mathbf{x}}_1 \\
\tilde{\mathbf{x}}_2 \\
\vdots \\
\tilde{\mathbf{x}}_{n-1} \\
\tilde{\mathbf{x}}_{n} 
\end{array}
\right] = 
\left[
\begin{array}{cccc}
x_{11} - \bar{x}_{11} & x_{12} - \bar{x}_{12} & \cdots & x_{1d} - \bar{x}_{1d} \\
x_{21} - \bar{x}_{21} & x_{22} - \bar{x}_{22} & \cdots & x_{2d} - \bar{x}_{2d} \\
\vdots & &\ddots & \vdots \\
x_{n1} - \bar{x}_{n1} & x_{n2} - \bar{x}_{n2} & \cdots & x_{nd} - \bar{x}_{nd} \\
\end{array}
\right]
\end{align*}

```
Exercise 1에서 생성한 RDD_step2 를 이용합니다.
 -> PythonRDD[86] at RDD at PythonRDD.scala:53 

Exercise 2에서 생성한 sample mean 을 이용합니다.
 -> [[427.88461538 186.32692308 344.05769231 178.05769231 129.48076923]]

#task 3-3 output
 -> [[-175.88461538 -111.32692308 -239.05769231 -124.05769231  -77.48076923]
 [ 451.11538462  240.67307692  236.94230769  113.94230769  148.51923077]
 [-284.88461538 -133.32692308 -272.05769231 -131.05769231  -98.48076923]
 [1072.11538462  595.67307692 1244.94230769  829.94230769  655.51923077]
 [-391.88461538 -174.32692308 -334.05769231 -174.05769231 -127.48076923]
 [-389.88461538 -174.32692308 -338.05769231 -176.05769231 -124.48076923], 
                                .
                                .
                                .
```

In [136]:
task3_3=RDD_step2.map(lambda x :(x-sample_mean))
print(task3_3)

PythonRDD[141] at RDD at PythonRDD.scala:53


In [41]:
print("Exercise 1에서 생성한 RDD_step2 를 이용합니다.\n ->",RDD_step2,"\n")
print("Exercise 2에서 생성한 sample mean 을 이용합니다.\n ->", sample_mean)

# 3 - 3 답안 작성
task3_3=RDD_step2.map(lambda x :(x-sample_mean)).reduce(lambda x,y:np.vstack((x,y)))#task3_3에 RDD_step2를map하고 x-sample_mean을 뺀 리스트를 받고 vstack을 통해 x,y로 행렬을 만들어줌
# output
print("\ntask 3-3 output\n ->", task3_3)

Exercise 1에서 생성한 RDD_step2 를 이용합니다.
 -> PythonRDD[35] at RDD at PythonRDD.scala:53 

Exercise 2에서 생성한 sample mean 을 이용합니다.
 -> [427.88461538 186.32692308 344.05769231 178.05769231 129.48076923]

task 3-3 output
 -> [[-175.88461538 -111.32692308 -239.05769231 -124.05769231  -77.48076923]
 [ 451.11538462  240.67307692  236.94230769  113.94230769  148.51923077]
 [-284.88461538 -133.32692308 -272.05769231 -131.05769231  -98.48076923]
 [1072.11538462  595.67307692 1244.94230769  829.94230769  655.51923077]
 [-391.88461538 -174.32692308 -334.05769231 -174.05769231 -127.48076923]
 [-389.88461538 -174.32692308 -338.05769231 -176.05769231 -124.48076923]
 [-412.88461538 -185.32692308 -341.05769231 -173.05769231 -129.48076923]
 [-336.88461538 -165.32692308 -308.05769231 -161.05769231 -115.48076923]
 [ 776.11538462  274.67307692  473.94230769  249.94230769  233.51923077]
 [ 146.11538462  181.67307692  256.94230769  237.94230769  119.51923077]
 [ 703.11538462  473.67307692  949.94230769  592.942307

**task**
4. Sample covariance matrix는 다음과 같이 정의됩니다 (15 point)
    - 행렬곱을 진행할 때, ``.T`` 또는 ``reshape``와 ``np.matmul``을 사용할 것!
\begin{align*}
\text{Cov}_2(\mathbf{x}) &= \frac{1}{n}\mathbf{X}^T \mathbf{X} \\
&= \left[
\begin{array}{cccc}
x_{11} - \bar{x}_{11} & x_{21} - \bar{x}_{21} & \cdots & x_{n1} - \bar{x}_{n1} \\
x_{12} - \bar{x}_{12} & x_{22} - \bar{x}_{22} & \cdots & x_{n2} - \bar{x}_{n2} \\
\vdots & &\ddots & \vdots \\
x_{1d} - \bar{x}_{1d} & x_{2d} - \bar{x}_{2d} & \cdots & x_{nd} - \bar{x}_{nd} \\
\end{array}
\right]\left[
\begin{array}{cccc}
x_{11} - \bar{x}_{11} & x_{12} - \bar{x}_{12} & \cdots & x_{1d} - \bar{x}_{1d} \\
x_{21} - \bar{x}_{21} & x_{22} - \bar{x}_{22} & \cdots & x_{2d} - \bar{x}_{2d} \\
\vdots & &\ddots & \vdots \\
x_{n1} - \bar{x}_{n1} & x_{n2} - \bar{x}_{n2} & \cdots & x_{nd} - \bar{x}_{nd} \\
\end{array}
\right]
\end{align*}

```
# task 3-4 output
Sample covariance matrix 방법2 결과(동일한 결과가 출력되어야 합니다)
 -> [[191231.67899408  92257.69156805 179270.89127219  94635.62204142
   70490.86316568]
 [ 92257.69156805  47359.72004438  90009.30806213  47427.92344675
   35666.34282544]
 [179270.89127219  90009.30806213 177880.51590237  95548.9966716
   70259.24149408]
 [ 94635.62204142  47427.92344675  95548.9966716   54126.05436391
   39050.81841716]
 [ 70490.86316568  35666.34282544  70259.24149408  39050.81841716
   29639.63424556]]

cov1과 cov2의 비교(오차)
 ->  0.0
 ```

In [51]:
a=[1,2]
b=[2,3]
np.matmul(a,b)

8

In [66]:
# 3-4 답안 작성
cov2=np.matmul(task3_3.T,task3_3)/RDD_step2.count()#cov2에 np.matmul(task3_3.t와 task3_3을 전치행렬로 만들고)/갯수를 나눠줌

# output
print("Sample covariance matrix 방법2 결과(동일한 결과가 출력되어야 합니다)\n ->", cov2)#cov2출력
print("\ncov1과 cov2의 비교(오차)\n -> ", sum((np.round(np.ravel(cov1), 2) - np.round(np.ravel(cov2), 2)) **2))

Sample covariance matrix 방법2 결과(동일한 결과가 출력되어야 합니다)
 -> [[191231.67899408  92257.69156805 179270.89127219  94635.62204142
   70490.86316568]
 [ 92257.69156805  47359.72004438  90009.30806213  47427.92344675
   35666.34282544]
 [179270.89127219  90009.30806213 177880.51590237  95548.9966716
   70259.24149408]
 [ 94635.62204142  47427.92344675  95548.9966716   54126.05436391
   39050.81841716]
 [ 70490.86316568  35666.34282544  70259.24149408  39050.81841716
   29639.63424556]]

cov1과 cov2의 비교(오차)
 ->  0.0


### Excercise 4 - 
#### 공분산 구하기 세 번째 방법을 적용해 보자! (25 point)

- - -

``Excercise1 에서 주어진 RDD_step2를 이용하여 진행``합니다.

- - -

#### 공분산 구하기 세번째 방법

1. Sample vector $\mathbf{x}_i$, $i=1,\ldots, n$와 평균 벡터 $\bar{\mathbf{x}}$ 에 대해서 다음 연산을 수행합니다. 
2. 각 벡터 $\mathbf{x}_i$의 outer product (외적)을 구합니다. (10 point)
\begin{align*}
\mathbf{O}_i = \mathbf{x}_i^T \mathbf{x}_i
\end{align*}
**(★주의★) 위에서와 같이 평균을 뺀 벡터가 아닙니다!**

<br>

3. 평균 벡터의 외적을 구합니다
\begin{align*}
\bar{\mathbf{O}} = \bar{\mathbf{x}}^T \bar{\mathbf{x}}
\end{align*}

<br>

4. Sample covariance matrix는 다음과 같이 구합니다
\begin{align*}
\text{Cov}_3(\mathbf{x}) &= \left(\frac{1}{n}\sum_{i=1}^n \mathbf{O}_i\right) - \bar{\mathbf{O}}
\end{align*}

**task**

1. Sample vector $\mathbf{x}_i$, $i=1,\ldots, n$와 평균 벡터 $\bar{\mathbf{x}}$ 에 대해서 다음 연산을 수행합니다. 
2. 각 벡터 $\mathbf{x}_i$의 outer product (외적)을 구합니다. (10 point)
\begin{align*}
\mathbf{O}_i = \mathbf{x}_i^T \mathbf{x}_i
\end{align*}
**(★주의★) 위에서와 같이 평균을 뺀 벡터가 아닙니다!**

<br>

3. 평균 벡터의 외적을 구합니다. (5 point)
\begin{align*}
\bar{\mathbf{O}} = \bar{\mathbf{x}}^T \bar{\mathbf{x}}
\end{align*}

```
Exercise 1에서 생성한 RDD_step2 를 이용합니다.
 -> PythonRDD[86] at RDD at PythonRDD.scala:53 

Exercise 2에서 생성한 sample mean 을 이용합니다.
 -> [[427.88461538 186.32692308 344.05769231 178.05769231 129.48076923]]

# task 4-3 output
 -> [[183085.24408284  79726.42381657 147216.9933432   76188.14718935
   55402.82914201]
 [ 79726.42381657  34717.72226331  64107.21116864  33176.94193787
   24125.7533284 ]
 [147216.9933432   64107.21116864 118375.69563609  61262.11871302
   44548.85465976]
 [ 76188.14718935  33176.94193787  61262.11871302  31704.54178994
   23055.04696746]
 [ 55402.82914201  24125.7533284   44548.85465976  23055.04696746
   16765.26960059]]
   ```

In [134]:
print("Exercise 1에서 생성한 RDD_step2 를 이용합니다.\n ->",RDD_step2,"\n")
print("Exercise 2에서 생성한 sample mean 을 이용합니다.\n ->", sample_mean)

# 4 - 2 답안 작성
sample_outer=RDD_step2.map(lambda x:np.outer(x,x))#outer를 통해 원소의전체를 외적해줌
sample_mean_outer=np.matmul(sample_mean.T,sample_mean)#np.matmul을 통해 평균 벡터구해준 것에서 전치행렬로 곱을 해줌



# output
print("\n평균 벡터의 외적\n ->", sample_mean_outer)#평균벡터의 외적 출력

Exercise 1에서 생성한 RDD_step2 를 이용합니다.
 -> PythonRDD[35] at RDD at PythonRDD.scala:53 

Exercise 2에서 생성한 sample mean 을 이용합니다.
 -> [[427.88461538 186.32692308 344.05769231 178.05769231 129.48076923]]

평균 벡터의 외적
 -> [[183085.24408284  79726.42381657 147216.9933432   76188.14718935
   55402.82914201]
 [ 79726.42381657  34717.72226331  64107.21116864  33176.94193787
   24125.7533284 ]
 [147216.9933432   64107.21116864 118375.69563609  61262.11871302
   44548.85465976]
 [ 76188.14718935  33176.94193787  61262.11871302  31704.54178994
   23055.04696746]
 [ 55402.82914201  24125.7533284   44548.85465976  23055.04696746
   16765.26960059]]


**task**

4. Sample covariance matrix는 다음과 같이 구합니다. (10 point)
\begin{align*}
\text{Cov}_3(\mathbf{x}) &= \left(\frac{1}{n}\sum_{i=1}^n \mathbf{O}_i\right) - \bar{\mathbf{O}}
\end{align*}

```
# task 4-4 output
Sample covariance matrix 방법3 결과(동일한 결과가 출력되어야 합니다)
 -> [[191231.67899408  92257.69156805 179270.89127219  94635.62204142
   70490.86316568]
 [ 92257.69156805  47359.72004438  90009.30806213  47427.92344675
   35666.34282544]
 [179270.89127219  90009.30806213 177880.51590237  95548.9966716
   70259.24149408]
 [ 94635.62204142  47427.92344675  95548.9966716   54126.05436391
   39050.81841716]
 [ 70490.86316568  35666.34282544  70259.24149408  39050.81841716
   29639.63424556]]

cov1과 cov3의 비교(오차)
 ->  0.0
```

In [137]:
# 4 - 4 답안 작성
cov3=sample_outer.reduce(lambda x,y:x+y)/RDD_step2.count()-sample_mean_outer#cov3에 reduce를 통해 모두 더해준 후 갯수로 나눠줌(시그마 1부터 n까지 더해주고 n/1로 나누는 과정)

# output
print("Sample covariance matrix 방법3 결과(동일한 결과가 출력되어야 합니다)\n ->", cov3)#cov3을 출력

print("\ncov1과 cov3의 비교(오차)\n -> ", 
     sum((np.round(np.ravel(cov1), 2) - np.round(np.ravel(cov3), 2)) **2))

Sample covariance matrix 방법3 결과(동일한 결과가 출력되어야 합니다)
 -> [[191231.67899408  92257.69156805 179270.89127219  94635.62204142
   70490.86316568]
 [ 92257.69156805  47359.72004438  90009.30806213  47427.92344675
   35666.34282544]
 [179270.89127219  90009.30806213 177880.51590237  95548.9966716
   70259.24149408]
 [ 94635.62204142  47427.92344675  95548.9966716   54126.05436391
   39050.81841716]
 [ 70490.86316568  35666.34282544  70259.24149408  39050.81841716
   29639.63424556]]

cov1과 cov3의 비교(오차)
 ->  0.0



 ![hi](http://www.filetalk.net/data/editor/1812/3b62f2a770e8c0fb20a6ee8d7368d27c_1545872435_993.gif)**수**
![hi2](https://images2.imgbox.com/91/ec/mGQ8RQCe_o.gif)**고**